# Optimization Techniques in Machine Learning

Objective: This assignment aims to explore implementation or Machine Learning Models with regularization, optimization and Error analysis  techniques used in machine learning to improve models' performance, convergence speed, and efficiency.

A Notebook detailing the following

* Project name
* Clear out puts from cells

**Instructions**
1. Acquire a dataset suitable for ML tasks as per your proposal.
2. Implement a simple machine learning model based on neural networks on the chosen dataset without any defined optimization techniques. (Check instructions)
3. Implement and compare the model's performance after applying 3 to 4 disntict combinations regularization and optimization techniques.
4. Discuss the results on the README file.
5. Make predictions using test data
7. Implement error analysis techniques and ensure there is: F1-Score, Recall, Precision, RUC a confusion matrix using plotting libraries (not verbose)

Submit notebook to github repo


---




# Libraries

In [ ]:
#Import Necessary Libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.callbacks import EarlyStopping
from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'pandas'

# The Dataset
> ***Brief Description:***
State the Problem and A short Description of the data

---

For this project, I'll use the Adult Income Dataset (also known as the "Census Income" dataset). This dataset contains information about individuals' income levels based on various features such as age, education, occupation, and more. The goal is to predict whether an individual's income exceeds $50,000 per year. The dataset can be found in the UCI Machine Learning Repository: [Adult Income Dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data).

In [ ]:
# Load dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"]
data = pd.read_csv(url, names=columns, sep=',\s', na_values=["?"], engine='python')

# Drop rows with missing values
data.dropna(inplace=True)

# Encode categorical variables
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Split data
X = data.drop('income', axis=1)
y = data['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Simple Neural Network Model (No Optimization Techniques)

In [ ]:
# Define a simple neural network model without optimization techniques
def simple_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train the simple model
simple_model = simple_model()
simple_history = simple_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Evaluate the simple model
simple_loss, simple_accuracy = simple_model.evaluate(X_test, y_test)
print(f"Simple Model Test Accuracy: {simple_accuracy}")

# Neural Network Models with Optimization Techniques

In [ ]:
# Define a function to create models with optimization techniques
def define_model(optimizer, regularization, early_stopping, dropout, learning_rate, layers):
    model = Sequential()
    for units in layers:
        model.add(Dense(units, activation='relu', kernel_regularizer=regularization))
        if dropout:
            model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Model 1: Adam optimizer, L2 regularization, Early stopping
model_1 = define_model(Adam, l2(0.01), True, 0.2, 0.001, [64, 32])
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history_1 = model_1.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Model 2: RMSprop optimizer, L1 regularization, No Early stopping
model_2 = define_model(RMSprop, l1(0.01), False, 0.2, 0.001, [64, 32])
history_2 = model_2.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

# Model 3: Custom combination
model_3 = define_model(Adam, l1(0.01), True, 0.3, 0.01, [128, 64, 32])
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history_3 = model_3.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Model 4: Another custom combination
model_4 = define_model(RMSprop, l2(0.01), True, 0.2, 0.001, [256, 128, 64])
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history_4 = model_4.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate and Compare Models

In [ ]:
# Evaluate each model on the test set
models = [simple_model, model_1, model_2, model_3, model_4]
histories = [simple_history, history_1, history_2, history_3, history_4]
model_names = ['Simple Model', 'Model 1', 'Model 2', 'Model 3', 'Model 4']

results = []
for model, history, name in zip(models, histories, model_names):
    loss, accuracy = model.evaluate(X_test, y_test)
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)
    f1 = f1_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    results.append((name, accuracy, f1, recall, precision, roc_auc))

# Print results
for result in results:
    print(f"Model: {result[0]}")
    print(f"Accuracy: {result[1]}")
    print(f"F1 Score: {result[2]}")
    print(f"Recall: {result[3]}")
    print(f"Precision: {result[4]}")
    print(f"ROC AUC: {result[5]}")
    print()

# Error Analysis

In [ ]:
# Plot confusion matrix for the best model
best_model = model_4
y_pred = best_model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Save Models

In [ ]:
simple_model.save('saved_models/simple_model.h5')
model_1.save('saved_models/model_1.h5')
model_2.save('saved_models/model_2.h5')
model_3.save('saved_models/model_3.h5')
model_4.save('saved_models/model_4.h5')